In [1]:
import jsonlines
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [2]:
with jsonlines.open('jsonl/train.jsonl') as reader:  
    train = [obj for obj in reader]
    
with jsonlines.open('jsonl/train-labels.lst') as reader:  
    train_labels = [obj for obj in reader] 
    
with jsonlines.open('jsonl/dev.jsonl') as reader:  
    dev = [obj for obj in reader]
    
with jsonlines.open('jsonl/dev-labels.lst') as reader:  
    dev_labels = [obj for obj in reader] 

In [3]:
pseudo_predictions = [1 for i in range(len(train))]
pseudo_predictions[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [4]:
train[100]

{'story_id': '002fa9df-ad31-49ef-bb25-d2d0144d6b7c-1',
 'obs1': 'The boy loved telling scary stories.',
 'obs2': 'She mad him stop telling myths.',
 'hyp1': 'His stories were so vivid it gave her nightmares .',
 'hyp2': "His stories were so fake it didn't give her nightmares."}

In [5]:
train_labels[100]

1

In [13]:
o1 = []
o2 = []
h = []
ll = []
for X,Y in zip(train,train_labels):
    o1.append(X['obs1'])
    o2.append(X['obs2'])
    h.append(X['hyp1'])
    ll.append(1 if Y==1 else 0)
    
    o1.append(X['obs1'])
    o2.append(X['obs2'])
    h.append(X['hyp2'])
    ll.append(1 if Y==2 else 0)


In [15]:
ll[:20]

[0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1]

In [17]:
train_labels[:10]

[2, 1, 1, 1, 1, 1, 2, 2, 1, 2]

In [12]:
def Precision(TP,FP):
    if TP == 0:
        return 0
    else:
        return TP/(TP+FP)

def Recall(TP,FN):
    if TP == 0:
        return 0
    else:
        return TP/(TP+FN)

def F(P,R):
    if P==0:
        return 0
    else:
        return 2*P*R/(P+R)
        
def Accuracy(TP, TN, FP, FN):
    return ( TP + TN )/ (TP + TN + FP + FN)
                               

def Eva(Hn, predictions, labels):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for p,l in zip(predictions,labels):
        if p == Hn:
            if l == p:
                TP += 1
            else:
                FP += 1
        else:
            if l == p:
                TN += 1
            else:
                FN += 1
    P = Precision(TP,FP)
    R = Recall(TP,FN)
    F1 = F(P,R)
    Acc = Accuracy(TP, TN, FP, FN)
    print("H%d: Precison %f, Recall %f, Accuracy %f, F1 %f" %(Hn, P, R, Acc, F1)) 

In [13]:
Eva(1, pseudo_predictions, train_labels)

H1: Precison 0.500029, Recall 1.000000, Accuracy 0.500029, F1 0.666693


In [14]:
Eva(2, pseudo_predictions, train_labels)

H2: Precison 0.000000, Recall 0.000000, Accuracy 0.500029, F1 0.000000


In [18]:
precision_recall_fscore_support(train_labels, pseudo_predictions, average=None)

/Users/wangtianxiang/miniconda3/envs/myenv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.50002947, 0.        ]),
 array([1., 0.]),
 array([0.66669286, 0.        ]),
 array([84832, 84822]))